## 환경설정

In [1]:
!git clone https://github.com/Quietseong/saju-prompt-mvp.git
%cd saju-prompt-mvp

fatal: destination path 'saju-prompt-mvp' already exists and is not an empty directory.
/content/saju-prompt-mvp


In [6]:
!pip install torch transformers accelerate python-dotenv
!pip install git+https://github.com/huggingface/transformers  # 최신 transformers 필요

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-n8_tthtd
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-n8_tthtd
  Resolved https://github.com/huggingface/transformers to commit aa27fa75cd05320dd6656fd9237f9647538a68b5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [7]:
!pip install -U bitsandbytes einops

In [8]:
!pip install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [3]:
from huggingface_hub import notebook_login
notebook_login()

## 단일 질문으로 테스트해보기

In [4]:
from src.gpt_client import get_llm_client
import bitsandbytes

llm = get_llm_client("llama3")
result = llm.generate_completion(
    prompt="1990-01-01 13:45 Seoul, please interpret the fortune of a san francisco woman.",
    temperature=0.7,
    max_tokens=128
)
print(result["text"])

DEBUG llm_type: llama3


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


What a fascinating request!

According to the ancient art of astrology, I shall interpret the fortune of a San Francisco woman born on January 1, 1990, at 13:45 (1:45 PM) in Seoul, South Korea.

**Sun Sign:** Capricorn
**Moon Sign:** Gemini
**Ascendant:** Leo

**Personality Insights:**

* Capricorn Sun: This woman is ambitious, responsible, and driven. She has high standards and is determined to succeed in her endeavors. Her practical and disciplined approach to life helps her achieve her goals.
* Gemini Moon: Her emotional nature is curious,


## 대화형으로 테스트해보기

In [5]:
from src.gpt_client import get_llm_client

llm = get_llm_client("llama3")
history = [
    {"role": "user", "content": "1990-01-01 13:45 Seoul , please interpret the fortune of a san francisco woman."}
]
# 첫 응답
result = llm.chat_with_history(history, temperature=0.7, max_tokens=128)
print(result["text"])

# 대화 이어가기
history.append({"role": "assistant", "content": result["text"]})
history.append({"role": "user", "content": "What more information do I need to enter?"})
result2 = llm.chat_with_history(history, temperature=0.7, max_tokens=128)
print(result2["text"])

DEBUG llm_type: llama3


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


What a fascinating request!

As I tune in to the energies of the universe, I see that on January 1st, 1990, at 13:45 in Seoul, South Korea, the celestial bodies are aligning to reveal the fortune of a San Francisco woman.

**Date:** January 1st, 1990, marks a fresh start, a new beginning, and an opportunity for this woman to embark on a new journey, leaving the past behind.

**Time:** 13:45 in Seoul, which is equivalent to 4:45 AM in San Francisco, suggests that this woman is likely to be an early
To provide a more accurate and personalized fortune, I would like to know a bit more information about the San Francisco woman. Please provide the following details:

1. Her name (or a nickname)
2. Her birth date and birth time (if known)
3. Her current age (or a range of ages, e.g., 25-35)
4. Her zodiac sign (if known) or her birth month (if not zodiac- savvy)
5. Any specific areas of life she'd like guidance on (e.g., relationships, career, finances, spirituality)

With this information, I'

In [6]:
# 3. 추가 질문
history.append({"role": "assistant", "content": result2["text"]})
history.append({"role": "user", "content": "Please explain in more detail about the sun sign."})
result3 = llm.chat_with_history(history, temperature=0.7, max_tokens=128)
print(result3["text"])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


I'd be happy to explain!

In astrology, the Sun Sign, also known as the Star Sign, is the zodiac sign associated with a person's birth date. The Sun Sign is calculated based on the date of birth, and it's considered a fundamental aspect of a person's birth chart.

Here's how it works:

* The Sun Sign is determined by the position of the Sun at the exact time and place of a person's birth.
* The zodiac wheel is divided into 12 segments, each representing a different sign, such as Aries, Taurus, Gemini, and so on.
* Each sign has its unique


In [8]:
# 4. 한 번 더 추가 질문
history.append({"role": "assistant", "content": result3["text"]})
history.append({"role": "user", "content": "My name is Emma. I was born on january 1, 1990 at 3:45 PM. and i'm 33 years old, I'd like guidance on my relationships"})
result4 = llm.chat_with_history(history, temperature=0.7, max_tokens=128)
print(result4["text"])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Emma!

As I tune in to the celestial energies, I see that your birth chart is a fascinating blend of elements!

**Sun Sign:** As you were born on January 1, 1990, at 3:45 PM, your Sun Sign is Capricorn. Capricorns are known for their ambition, discipline, and responsible nature. You're likely a hard worker, dedicated to your goals, and value stability and security.

**Relationships:** Ah, your question about relationships! As a Capricorn, you tend to approach relationships with a practical, logical, and responsible mindset. You value commitment, loyalty, and


## 챗봇 데모

In [2]:
# 1. 필수 패키지 설치 (Colab에서 최초 1회만)
!pip install gradio transformers torch bitsandbytes einops

# 2. (필요시) HuggingFace 토큰 인증 (Llama-3 등 gated 모델 사용 시)
from huggingface_hub import notebook_login
notebook_login()

# 3. (필요시) 코드/모델 파일 업로드 또는 GitHub에서 clone
# 예시: GitHub에서 프로젝트 전체 clone
!git clone https://github.com/your-username/your-repo.git
%cd your-repo

# 4. Gradio 챗봇 실행 코드
import gradio as gr
from src.gpt_client import get_llm_client

llm = get_llm_client("llama3")  # 또는 "openai", "sllm" 등

def chat_fn(message, history):
    llm_history = []
    for user_msg, assistant_msg in history:
        llm_history.append({"role": "user", "content": user_msg})
        if assistant_msg:
            llm_history.append({"role": "assistant", "content": assistant_msg})
    llm_history.append({"role": "user", "content": message})
    result = llm.chat_with_history(llm_history, temperature=0.7, max_tokens=256)
    return result["text"]

demo = gr.ChatInterface(
    fn=chat_fn,
    title="Llama-3 사주 챗봇 데모",
    description="사주, 운세, 별자리 등 다양한 질문을 해보세요!",
    examples=[
        ["1990-01-01 13:45 SF , please interpret the fortune of a san francisco woman."],
        ["What more information do I need to enter?"],
        ["My name is Emma. I was born on january 1, 1990 at 3:45 PM. and i'm 33 years old, I'd like guidance on my relationships"]
    ]
)

demo.launch(share=True)

Cloning into 'your-repo'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: 'your-repo'
/content/saju-prompt-mvp
DEBUG llm_type: llama3


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://301a40c9c0b438fca8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
